In [8]:
import quantumpropagator as qp
import matplotlib.pyplot as plt
import matplotlib
from ipywidgets import interact,fixed #, interactive, fixed, interact_manual
import ipywidgets as widgets

import os
import sys
import glob
import pandas as pd
import numpy as np

%matplotlib ipympl

plt.rcParams.update({'font.size': 10})

In [9]:
fol = '/home/alessio/w-August-Run/'
subfolders = sorted([dir for dir in os.listdir(fol) if os.path.isdir(os.path.join(fol,dir)) and dir != os.path.join(fol,'.ipynb_checkpoints')])
print(''.join(['{} -> {}\n'.format(a,b) for a,b in enumerate(subfolders)]))

0 -> HTML
1 -> b-UV-0.22_0000
2 -> csv
3 -> m-IR-Polarized-INV_0000
4 -> m-IR-Polarized-LON_0000
5 -> m-IR-Polarized-NOR_0000
6 -> position-from1_0000
7 -> position-from2_0000
8 -> position-from3_0000
9 -> position-from4_0000
10 -> position-from5_0000
11 -> t-FC_excited_phi_0000
12 -> t-IR-to-excite-phi_0000
13 -> t-UV-0.22_excited_phi_0001
14 -> z-from1_0000



In [15]:

def process_folder(this_folder):
    '''
    this function will create a correct set of files to give to Francoise
    '''
    if this_folder == 'HTML' or this_folder == 'csv':
        sys.exit('no')
    return(this_folder)

process_folder(subfolders[0])

SystemExit: no

/home/alessio/config/miniconda/envs/quantumpropagator/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
